In [5]:
import rustworkx as rx

from itertools import product

from territories import Territory
from territories.partitions import TerritorialUnit, Partition


lyon = TerritorialUnit("Lyon")
marseille = TerritorialUnit("Marseille", es_code="COM:2909") # you can specify an ElasticSearch code
paris = TerritorialUnit("Paris")
nogent = TerritorialUnit("Nogent")
pantin = TerritorialUnit("Pantin")
villeurbane = TerritorialUnit("Villeurbane")
sté = TerritorialUnit("Saint Etienne")

metropole = TerritorialUnit("Grand Lyon", False, Partition.DEP)

sud = TerritorialUnit("Sud", False, Partition.REGION)
idf = TerritorialUnit("Île-de-France", False, Partition.REGION)
rhone = TerritorialUnit("Rhône", False, Partition.DEP)

france = TerritorialUnit("France", False, Partition.COUNTRY)



entities = (france, sud, idf, rhone, metropole, nogent, pantin, paris, marseille, sté, villeurbane, lyon)

tree = rx.PyDiGraph()
entities_indices = tree.add_nodes_from(entities)

mapper = {o : idx for o, idx in zip(entities, entities_indices)}
edges = [
    (france, idf),
    (france, sud),
    
    (idf, nogent),
    (idf, pantin),
    (idf, paris),

    (sud, marseille),
    (sud, rhone),

    (rhone, metropole),
    (rhone, sté),

    (metropole, villeurbane),
    (metropole, lyon),
    ]

tree.add_edges_from([
    (mapper[parent], mapper[child], None) for parent, child in edges
])

Territory.assign_tree(tree)

a = Territory(sté, marseille)
b = Territory(lyon, france)
c = Territory(paris, nogent, pantin, lyon, lyon, metropole)
d = Territory(lyon, villeurbane, marseille)
e = Territory(rhone, idf)
f = Territory(idf, marseille, metropole)

exemples = (a, b, c, d, e, f)

In [12]:
import os
import pickle
import rustworkx as rx

from territories import Partition

from pathlib import Path


path = Path(os.environ["API_CACHE_DIR"], "territorial_tree_state.pickle")

with open(path, "rb") as file:
    _, tree = pickle.load(file)

comms = [t.es_code + '\n' for t in tree.nodes() if t.partition_type == Partition.COM]

with open("comms.txt", "w") as file:
    file.writelines(comms)